In [6]:
import pandas as pd
df=pd.read_csv("merged_barrier_bianeu_llmhuman.csv")

#drop rows which 'source'=human
df=df[df['source']!='human']

#save to csv
df.to_csv("merged_barrier_biasneu_no_human.csv",index=False)

#change where 'condition'=BIASED to 'ANCHOR'
df['condition']=df['condition'].replace('BIASED','ANCHOR')

#save to csv
df.to_csv("merged_barrier_anchorneu-no_human.csv",index=False)

#change where 'condition'='NEU' to 'CONTEXT'
df['condition']=df['condition'].replace('NEU','CONTEXT')

#save to csv
df.to_csv("merged_barrier_anchorctx-no_human.csv",index=False)

#change where 'scenario'=BIASED to 'ANCHOR'
df['scenario']=df['scenario'].replace('BIASED','ANCHOR')

#save to csv
df.to_csv("merged_barrier_anchorneu-scenarioanchor.csv",index=False)

#change where 'scenario'='NEU' to 'CONTEXT'
df['scenario']=df['scenario'].replace('NEU','CONTEXT')

#save to csv
df.to_csv("merged_barrier_anchorctx-scenario_anchorcontext.csv",index=False)

df.shape

(10800, 13)

In [9]:
import pandas as pd
# csv
df=pd.read_csv("merged_barrier_anchorctx-scenario_anchorcontext.csv")


#change 'bias_type' from 'BIAS_*'   to 'ANCHOR_*'
df['bias_type']=df['bias_type'].replace('BIAS_WORD','ANCHOR_WORD')
df['bias_type']=df['bias_type'].replace('BIAS_EXAMPLE','ANCHOR_EXAMPLE')
df['bias_type']=df['bias_type'].replace('BIAS_NUM_LOW','ANCHOR_NUM_LOW')
df['bias_type']=df['bias_type'].replace('BIAS_NUM_HIGH','ANCHOR_NUM_HIGH')

#save to csv
df.to_csv("merged_barrier_anchorctx-scenario_anchortypes.csv",index=False)

In [10]:
import pandas as pd
import re
# csv
df=pd.read_csv("merged_barrier_anchorctx-scenario_anchortypes.csv")

mapping = {
    "BIAS_WORD":     "ANCHOR_WORD",
    "BIAS_EXAMPLE":  "ANCHOR_EXAMPLE",
    "BIAS_NUM_LOW":  "ANCHOR_LOW_NUM",
    "BIAS_NUM_HIGH": "ANCHOR_HIGH_NUM",
}

# Plain string
def replace_bias_prefix(s: str) -> str:
    return re.sub(
        r"^(BIAS_WORD|BIAS_EXAMPLE|BIAS_NUM_LOW|BIAS_NUM_HIGH)",
        lambda m: mapping[m.group(1)],
        s,
    )

# Pandas column
df['row_id'] = df['row_id'].apply(replace_bias_prefix)

#save to csv
df.to_csv("merged_barrier_anchorctx-scenario_anchortypes_rowid_fixed.csv", index=False)


In [ ]:
import pandas as pd
import numpy as np
import textwrap
import re

df=pd.read_csv("merged_barrier_anchorctx-scenario_anchortypes_rowid_fixed.csv")

# --- Canonical mapping (your source of truth) ---
BARRIERS = {
    1: "power losses, quality and safety issues",
    2: "reduced reliability in DC devices",
    3: "lack of use-cases in which DC is advantageous",
    4: "uncertain utility interaction (net metering, utility ownership, and agreed standards)",
    5: "lack of pilot projects",
    6: "public perception of DC and readiness to 'champion' installations from DC projects",
    7: "incompatibility of DC systems components",
    8: "misconception and lack of knowledge leads to lengthy/expensive design and permit process",
    9: "lack of enough trained personnel in DC systems",
    10: "uncertain regulatory roadmap",
    11: "high costs of DC solutions",
}

def _norm(s: str) -> str:
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return ""
    s = str(s).lower().strip()
    s = re.sub(r"\s+", " ", s)                     # collapse whitespace
    s = re.sub(r"[^\w\s']", " ", s)                # drop punctuation except apostrophes
    s = re.sub(r"\s+", " ", s).strip()
    return s

# --- Main validator ---
def validate_barriers(df: pd.DataFrame,
                      id_col: str = "barrier_id",
                      text_col: str | None = "barrier_label") -> pd.DataFrame:
    """
    Checks:
      1) All IDs present in df are valid keys of BARRIERS.
      2) (Optional) If text_col is provided, compare normalized text to canonical text.
    Returns a per-row report with flags.
    """
    # Coerce IDs to integers where possible
    ids = pd.to_numeric(df[id_col], errors="coerce").astype("Int64")
    rep = df.copy()
    rep[id_col] = ids

    # Map expected canonical text
    canon_df = pd.DataFrame.from_dict(BARRIERS, orient="index", columns=["expected_text"]).rename_axis(id_col).reset_index()
    rep = rep.merge(canon_df, on=id_col, how="left")

    # Flags
    rep["id_known"] = rep["expected_text"].notna()

    if text_col is not None and text_col in rep.columns:
        rep["given_text_norm"]   = rep[text_col].map(_norm)
        rep["expected_text_norm"]= rep["expected_text"].map(_norm)
        rep["text_matches"]      = np.where(rep["id_known"], rep["given_text_norm"] == rep["expected_text_norm"], pd.NA)
    else:
        rep["text_matches"] = pd.NA

    # Summary prints
    invalid_ids = rep.loc[~rep["id_known"], id_col].dropna().unique()
    if len(invalid_ids):
        print(f"❌ Unknown barrier_id values found: {sorted(map(int, invalid_ids))}")
    else:
        print("✅ All barrier_id values are valid.")

    if text_col and text_col in rep.columns:
        mism = rep[(rep["id_known"]) & (rep["text_matches"] == False)]
        if len(mism):
            print(f"❌ {len(mism)} rows where {text_col} doesn't match the canonical text.")
        else:
            print(f"✅ All known IDs have matching {text_col} (after normalization).")

    # Useful quick stats
    print("\nCounts by barrier_id in your data:")
    print(rep["barrier_id"].value_counts(dropna=False).sort_index())

    return rep
# Run the validator (adjust column names if needed)
report = validate_barriers(df, id_col="barrier_id", text_col="barrier_text")

# Save full report
report.to_csv("barrier_validation_report.csv", index=False)

# (Optional) Save only problematic rows
bad_rows = report[(report["id_known"] == False) | (report["text_matches"] == False)]
bad_rows.to_csv("barrier_validation_issues.csv", index=False)

# (Optional) quick peek in notebook
report.head()

In [ ]:
import pandas as pd
df=pd.read_csv("merged_barrier_anchorctx-scenario_anchortypes_rowid_fixed.csv")

df.info()
print(df)

In [15]:
import pandas as pd
import numpy as np
df=pd.read_csv("merged_barrier_anchorctx-scenario_anchortypes_rowid_fixed.csv")
# ---- Canonical barriers ----
BARRIERS = {
    1: "power losses, quality and safety issues",
    2: "reduced reliability in DC devices",
    3: "lack of use-cases in which DC is advantageous",
    4: "uncertain utility interaction (net metering, utility ownership, and agreed standards)",
    5: "lack of pilot projects",
    6: "public perception of DC and readiness to 'champion' installations from DC projects",
    7: "incompatibility of DC systems components",
    8: "misconception and lack of knowledge leads to lengthy/expensive design and permit process",
    9: "lack of enough trained personnel in DC systems",
    10: "uncertain regulatory roadmap",
    11: "high costs of DC solutions"
}
valid_ids = set(BARRIERS.keys())

EXPECTED_PER_ITER_ANCHOR = 5  # anchors must be exactly 5
MAX_PER_ITER_CONTEXT     = 5  # context must be <= 5

# ---- df should already be loaded; if not: df = pd.read_csv("your_file.csv")
df = df.copy()

# Normalize key fields
df["base_model"] = df["base_model"].astype(str).str.strip()
df["variant_id"] = df["variant_id"].astype(str).str.strip()

# condition can be 'ANCHOR' or 'CONTEXT'
df["condition"] = df["condition"].astype(str).str.strip().str.upper()

# bias_type holds the anchor arm (if present)
if "bias_type" in df.columns:
    df["bias_type"] = df["bias_type"].astype(str).str.strip().str.upper().replace({"": np.nan})
else:
    df["bias_type"] = np.nan

# iteration numeric
df["iteration"] = pd.to_numeric(df["iteration"], errors="coerce").astype("Int64")

# barrier_id numeric Int64
df["barrier_id"] = pd.to_numeric(df["barrier_id"], errors="coerce").astype("Int64")

# Build the arm for grouping: CONTEXT vs anchor arm
# If condition==CONTEXT -> "CONTEXT"
# Else use bias_type if available; otherwise fallback to condition (will pool anchors together if bias_type missing)
df["arm"] = np.where(
    df["condition"] == "CONTEXT",
    "CONTEXT",
    np.where(df["bias_type"].notna(), df["bias_type"], df["condition"])
)

GROUP_COLS = ["base_model", "variant_id", "arm", "iteration"]

# ---- Per-group aggregation
grp = (
    df.groupby(GROUP_COLS, dropna=False)
      .agg(
          rows=("row_id", "size"),
          n_unique_ids=("barrier_id", lambda s: s.dropna().nunique()),
          any_na_ids=("barrier_id", lambda s: s.isna().any()),
          ids_list=("barrier_id", lambda s: tuple(int(x) for x in s.dropna().tolist())),
      )
      .reset_index()
)

# rule per group (vectorized)
is_context = grp["arm"].eq("CONTEXT")
is_anchor  = ~is_context

# Basic constraints
count_ok = pd.Series(False, index=grp.index)
uniq_ok  = pd.Series(False, index=grp.index)
valid_ok = grp["ids_list"].map(lambda ids: all(i in valid_ids for i in ids))

# CONTEXT: rows <= 5 and unique == rows (no duplicates)
count_ok[is_context] = grp.loc[is_context, "rows"] <= MAX_PER_ITER_CONTEXT
uniq_ok[is_context]  = grp.loc[is_context, "n_unique_ids"] == grp.loc[is_context, "rows"]

# ANCHOR arms: rows == 5 and unique == 5
count_ok[is_anchor] = grp.loc[is_anchor, "rows"] == EXPECTED_PER_ITER_ANCHOR
uniq_ok[is_anchor]  = grp.loc[is_anchor, "n_unique_ids"] == EXPECTED_PER_ITER_ANCHOR

grp["count_ok"]    = count_ok
grp["uniq_ok"]     = uniq_ok
grp["valid_ids_ok"]= valid_ok
grp["all_ok"]      = grp["count_ok"] & grp["uniq_ok"] & grp["valid_ids_ok"]

# Helpful diagnostics
def diag_row(r) -> str:
    msgs = []
    if r["arm"] == "CONTEXT":
        if not r["count_ok"]:
            msgs.append(f"CONTEXT count={r['rows']} (max {MAX_PER_ITER_CONTEXT})")
        if not r["uniq_ok"]:
            msgs.append(f"CONTEXT duplicate barrier_ids (unique={r['n_unique_ids']}, rows={r['rows']})")
    else:
        if not r["count_ok"]:
            msgs.append(f"ANCHOR count={r['rows']} (expected {EXPECTED_PER_ITER_ANCHOR})")
        if not r["uniq_ok"]:
            msgs.append(f"ANCHOR unique_ids={r['n_unique_ids']} (expected {EXPECTED_PER_ITER_ANCHOR})")
    if not r["valid_ids_ok"]:
        bad = [i for i in r["ids_list"] if i not in valid_ids]
        if bad:
            msgs.append(f"invalid_ids={bad}")
        else:
            msgs.append("invalid_ids=True")
    return "; ".join(msgs) or "OK"

violations_summary = (
    grp[~grp["all_ok"]]
    .assign(issue=lambda x: x.apply(diag_row, axis=1))
    .sort_values(GROUP_COLS)
)

# ---- Attach flags back to rows
df_flagged = df.merge(grp, on=GROUP_COLS, how="left", validate="many_to_one")

verified_rows   = df_flagged[df_flagged["all_ok"]].copy()
violations_rows = df_flagged[~df_flagged["all_ok"]].copy()

# ---- Save outputs
verified_rows.to_csv("barriers_verified_rows.csv", index=False)
violations_rows.to_csv("barriers_violations_rows.csv", index=False)
grp.to_csv("barriers_group_summary.csv", index=False)
violations_summary.to_csv("barriers_violations_summary.csv", index=False)

print("✅ Saved:")
print(" - barriers_verified_rows.csv (rows in groups that passed)")
print(" - barriers_violations_rows.csv (rows in groups that failed)")
print(" - barriers_group_summary.csv (one line per group with flags)")
print(" - barriers_violations_summary.csv (failed groups with diagnostics)")

# (Optional) sanity checks for your inventory: 4 base models × 6 variants each
inventory = df[["base_model", "variant_id"]].drop_duplicates()
inv_counts = inventory.groupby("base_model")["variant_id"].nunique().sort_index()
print("\nVariants per base_model:\n", inv_counts.to_string())


C:\Users\HP\AppData\Local\Temp\ipykernel_4944\3620546462.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[True True True ... True True True]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  count_ok[is_context] = grp.loc[is_context, "rows"] <= MAX_PER_ITER_CONTEXT


✅ Saved:
 - barriers_verified_rows.csv (rows in groups that passed)
 - barriers_violations_rows.csv (rows in groups that failed)
 - barriers_group_summary.csv (one line per group with flags)
 - barriers_violations_summary.csv (failed groups with diagnostics)

Variants per base_model:
 base_model
gemma3     12
llama      12
mistral    12
phi4       12


In [17]:
import pandas as pd
df=pd.read_csv("merged_barrier_anchorctx-scenario_anchortypes_rowid_fixed.csv")
#count rows which have condition as 'context'
print(df[df['condition'] == 'CONTEXT'].shape[0])

6000


In [20]:
import pandas as pd
df=pd.read_csv("merged_barrier_anchorctx_final.csv")
#generate new csv with only rows where condition is 'anchor'
df[df['condition'] == 'ANCHOR'].to_csv("orderedbarrier_anchor.csv", index=False)


In [23]:
import pandas as pd
df=pd.read_csv("orderedbarrier_anchor.csv")
df.shape

(4800, 13)

**this is reserved part for correcting IDs of barriers** so as to save a new csv with clean barrier id ready for anaylsis.

GUIDE: 
The full dataset with all corrected values is in:

barriers_corrected_by_text.csv

That file contains every original column plus:

barrier_correct_id — the ID inferred from official_label

barrier_correct_text — canonical text for that ID

(optional) id_changed — True where the original barrier_id ≠ barrier_correct_id

The other files are just reports:

barriers_unmatched_official_label.csv → rows where official_label didn’t match any dictionary entry (so barrier_correct_id is NA)

barriers_id_mismatch.csv → rows where original ID disagrees with text-derived ID

In [ ]:
import pandas as pd
import numpy as np
import re

# --- Canonical mapping (source of truth) ---
BARRIERS = {
    1: "power losses, quality and safety issues",
    2: "reduced reliability in DC devices",
    3: "lack of use-cases in which DC is advantageous",
    4: "uncertain utility interaction (net metering, utility ownership, and agreed standards)",
    5: "lack of pilot projects",
    6: "public perception of DC and readiness to 'champion' installations from DC projects",
    7: "incompatibility of DC systems components",
    8: "misconception and lack of knowledge leads to lengthy/expensive design and permit process",
    9: "lack of enough trained personnel in DC systems",
    10: "uncertain regulatory roadmap",
    11: "high costs of DC solutions",
}

# --- Lightweight normalizer so small formatting differences don't block matches ---
def _norm(s: str) -> str:
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return ""
    s = str(s).lower().strip()
    s = re.sub(r"\s+", " ", s)                 # collapse whitespace
    s = re.sub(r"[^\w\s']+", " ", s)           # drop punctuation except apostrophes
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Reverse map: normalized canonical text -> id
rev_map = { _norm(txt): bid for bid, txt in BARRIERS.items() }

# Assume you already loaded your CSV as df, and it has a column 'official_label'
# If your column name differs, change 'official_label' below accordingly.

# 1) Compute corrected id from official_label text
df["barrier_correct_id"] = df["official_label"].map(lambda s: rev_map.get(_norm(s), pd.NA)).astype("Int64")

# 2) (Optional) canonical text for that id, and quick comparison flags
df["barrier_correct_text"] = df["barrier_correct_id"].map(BARRIERS)
if "barrier_id" in df.columns:
    # Flag rows where the original id disagrees with the text-derived id
    df["barrier_id"] = pd.to_numeric(df["barrier_id"], errors="coerce").astype("Int64")
    df["id_changed"] = (df["barrier_id"] != df["barrier_correct_id"])

# 3) Save a full corrected CSV (all original columns + new columns)
out_csv = "barriers_corrected_by_text.csv"
df.to_csv(out_csv, index=False)
print(f"✅ Wrote: {out_csv}")

# 4) (Optional) Save only rows that could not be matched or that changed
unmatched = df[df["barrier_correct_id"].isna()]
changed   = df["id_changed"].fillna(False)
if len(unmatched):
    unmatched.to_csv("barriers_unmatched_official_label.csv", index=False)
    print(f"⚠️  Unmatched rows by text: {len(unmatched)} (saved barriers_unmatched_official_label.csv)")
if "id_changed" in df and changed.any():
    df.loc[changed].to_csv("barriers_id_mismatch.csv", index=False)
    print(f"⚠️  ID/text mismatches: {changed.sum()} (saved barriers_id_mismatch.csv)")


In [2]:
import pandas as pd
df=pd.read_csv("merged_barrier_anchorctx_final.csv")

#select unique values at 'condition' column
print(df['condition'].unique())


['ANCHOR' 'CONTEXT']
